# Representative period processing

This script processes representative periods for the NE-model.
The processing consists of the following steps:
1. Python configuration for the script.
2. Read `.gdx` time series from a pre-processed NE-model input folder.
3. Format them into a single massive dataframe for TSAM.
4. Run TSAM and process the representatives into Backbone samples.
5. Output Backbone settings.

You will need to have an input folder processed by the NE-model `building_input_data.py`,
as this scrip will use the preprocessed `.gdx` time series files.

## 1. Config

Import the necessary packages etc.

In [ ]:
## Import necessary packages

import os # For gathering all timeseries files.
from itertools import product, zip_longest # More efficient looping
import gams.transfer as gt # For reading said timeseries files.
import pickle # Save TSAM formatted data for reuse.
import tsam.timeseriesaggregation as tsam # For aggregating representatives.
import tsam.hyperparametertuning as hype # For testing optimizing representatives.
from datetime import datetime

In [ ]:
## Force data processing rerun if desired.

force_rerun_data = False # Define whether to force data processing rerun. Set this to `True` if you're uncertain.
rerun_data = (force_rerun_data or not os.path.exists("tsam_df_dict.pkl")) # Rerun data if forced or no previous data.

## 2. Read time series input data

Here, we read the time series input data from the given `input_folder_path`.
The `omit_suffixes`, `aggregate_weather_years` and `aggregate_timeseries_names` filters are also applied
here to avoid unnecessary input data reading.

In [ ]:
## Configure input data to be included for aggregation.

# Path to the NE-model preprocessed input data folder.
# The one containing a ton of ts_*.gdx files)
input_folder_path = "./north_european_model/input_National_Trends_2040_nucTypical/"

# Select weather years to be processed `list(<str>)`!!!
# `None` processes all available years.
aggregate_weather_years = None

# Select time series names for aggregation `list(<str>)`.
# `None` processes all available timeseries.
aggregate_timeseries_names = None

# Omit `.gdx` files with these as their suffix.
omit_suffixes = ["forecasts", "demands"]

In [ ]:
## Gather and filter the timeseries files in the input folder.

# Reading and preprocessing relevant filenames.
timeseries_files = os.listdir(input_folder_path) # Gather all files in the dir
timeseries_files = [f.split('.') for f in timeseries_files] # Split file suffix
timeseries_files = [f[0].split('_') for f in timeseries_files if f[-1] == "gdx"] # Split filename by '_' and filter by .gdx
timeseries_files = [('_'.join(f[0:-1]), f[-1]) for f in timeseries_files if f[0] == "ts"] # Form filenames/years and filter by 'ts' prefix

# Config filtering by suffix, timeseries names, and weather years.
if omit_suffixes is not None:
    timeseries_files = [f for f in timeseries_files if f[-1] not in omit_suffixes]
if aggregate_weather_years is not None:
    timeseries_files = [f for f in timeseries_files if f[-1] in aggregate_weather_years]
if aggregate_timeseries_names is not None:
    timeseries_files = [f for f in timeseries_files if f[0] in aggregate_timeseries_names]

# Determine sets of remaining filenames and years.
filtered_filenames = set([f[0] for f in timeseries_files])
filtered_years = set([f[-1] for f in timeseries_files])

In [ ]:
## Read .gdx data into a nested dictionary with `year`->`param_name` as the keys.

gdx_df_dict = dict() # Initialize empty dict for collecting input data.
if rerun_data:
    for (year, filename) in product(filtered_years, filtered_filenames):
        gdx = gt.Container(f"{input_folder_path}{filename}_{year}.gdx") # Read input data file.
        gdx_df_dict.setdefault(year, dict()) # Initialize empty sub-dictionary for parameter values.
        for (param_name, vals) in gdx.data.items():
            if param_name.startswith('ts_'): # Only record actual timeseries parameters, we're not interested in the dimensions.
                gdx_df_dict[year][param_name] = vals.records # Record values per year per param_name

## 3. Format data for TSAM

All the data needs to be in a single dataframe with timesteps as indices for TSAM,
while all timeseries values need to be pivoted to columns.

>**NOTE!**
>Currently, all timeseries are used raw, as-is, without any weighting.
>Better results might be achieved via some form of weighting, but I'm unsure how that should be implemented in TSAM.

In [ ]:
## Settings for TSAM data formatting
# Don't change these unless you know what you're doing.

index_column_name = 't' # Name of the time index column.
value_column_name = 'value' # Name of the value column.
forecast_filter = {'f00'} # Filter out forecasts besides these.

In [ ]:
## Format data for TSAM
# This unfortunately seems to take ~1.5 min for the full dataset,
# which is way longer than it takes to deduce the representative periods
# (at least employing the `hierarchical` method)

if rerun_data:
    tsam_df_dict = dict() # Initialize empty dict for collecting all timeseries per year.
    for (year, param_dict) in gdx_df_dict.items():
        for (param_name, vals) in param_dict.items():
            agg_cols = vals.columns.difference([index_column_name, value_column_name]) # Gather all other column names to be aggregated.
            agg_col_name = '-'.join([param_name, *agg_cols]) # Name for the aggregated column including parameter name.
            if vals.get('f') is not None: # Forecast filtering applied when needed.
                vals = vals.loc[vals['f'].isin(forecast_filter)]
            vals[agg_col_name] = param_name + '-' + vals[agg_cols].agg('-'.join, axis=1) # Create aggregate column value.
            vals = vals[[index_column_name, value_column_name, agg_col_name]] # Omit old columns.
            vals = vals.pivot( # Pivot timeseries dataframe for TSAM
                index=index_column_name, columns=agg_col_name, values=value_column_name
            )
            if tsam_df_dict.get(year) is None:
                tsam_df_dict[year] = vals # First dataframe becomes the starting point.
            else:
                tsam_df_dict[year] = tsam_df_dict[year].join(vals) # Rest are joined on index.
    with open("tsam_df_dict.pkl", "wb") as file: # Save data for future use.
        pickle.dump(tsam_df_dict, file)
else:
    with open("tsam_df_dict.pkl", "rb") as file: # Load previous saved data.
        tsam_df_dict = pickle.load(file)

## 4. Run TSAM and process Backbone samples.

The most important settings below are `noTypicalPeriods` and `hoursPerPeriod`,
maybe followed by `extremePeriodMethod` and `clusterMethod`.
The rest don't really need to be touched, unless you know what you're doing.

The full description of the TSAM parameters is not reproduced here,
please refer to their [documentation](https://tsam.readthedocs.io/en/latest/timeseriesaggregationDoc.html) (somewhat technical as it is).
Some of the concepts are more intuitively explained by their
[recent methodology paper](https://doi.org/10.1016/j.apenergy.2022.119029).

In [ ]:
## Configure TSAM aggregation
# (see https://tsam.readthedocs.io/en/latest/timeseriesaggregationDoc.html)

# Main settings you might be interested in tweaking.
noTypicalPeriods = 4 # Number of representative periods. (Periods for hypertuning)
hoursPerPeriod = 24 # Hours per representative period. (Segments for hypertuning?)
extremePeriodMethod = "replace_cluster_center" # Method to integrate extreme periods? Honestly, this doesn't seem to impact much?
clusterMethod = "hierarchical" # Select clustering method. (`hierarchical` or `k_medoids` recommended)

# Auxiliary settings you shoudldn't need to touch.
resolution = 1 # Resolution of input data in hours, shouldn't need to be touched.
rescaleClusterPeriods = False # Don't rescale periods, we don't use that data anyhow.
segmentation = True # Segmentation to enable hypertuning, doesn't impact the clustering results.
numericalTolerance = 1e-6 # Set numerical tolerance for TSAM so it stops complaining (doesn't seem to work for some reason).

In [ ]:
## TSAM time series aggregation
# This seems amazingly fast, less than 5 seconds for all the data.

# Suppress warnings from TSAM (because `numericalTolerance` doesn't seem to work for some reason.)
import warnings
warnings.filterwarnings("ignore")

tsam_dict = dict() # Initialize dict to store TSAM aggregation per year.
for (year, data) in tsam_df_dict.items():
    aggregation = tsam.TimeSeriesAggregation( ## Define TSAM aggregation.
        data,
        noTypicalPeriods=noTypicalPeriods,
        hoursPerPeriod=hoursPerPeriod,
        clusterMethod=clusterMethod,
        resolution=resolution,
        rescaleClusterPeriods=rescaleClusterPeriods,
        extremePeriodMethod=extremePeriodMethod,
        segmentation=segmentation,
        numericalTolerance=numericalTolerance,
    )
    aggregation.createTypicalPeriods() ## Run TSAM aggregation.
    tsam_dict[year] = aggregation

# Re-enable warnings
warnings.filterwarnings("default")

In [ ]:
## Optional hypertuning test
# Demoes hypertuning functionality with NE-data.

hypertuning_year = None # Set a year for hypertuning
if hypertuning_year is not None:
    # Suppress warnings
    import warnings
    warnings.filterwarnings('ignore')
    # Data reduction factor calculated based on given aggregation settings
    reduction_factor = (
        noTypicalPeriods * hoursPerPeriod / len(tsam_df_dict[hypertuning_year])
    )
    # Run hypertuning
    hyper = hype.HyperTunedAggregations(tsam_dict[year])
    result = hyper.identifyOptimalSegmentPeriodCombination(reduction_factor)
    print(result)
    warnings.filterwarnings('default')

# E.g. apparently for 1990-1991 and 2000, 44 periods of 15 hours (44x15 = 660)
# should be used instead of 4 weeks (4*168 = 672).

# See `ne-model-hypertuning.ipynb` for a more thorough analysis.

In [ ]:
## Process Backbone samples per year from the aggregation
# The samples are processed based on the "clusters" in TSAM.
# NOTE! There is no guarantee that TSAM samples occur in a sequence similar to Backbone samples!
# However, based on very brief testing this seems to mostly be the case? 

sample_dict = dict() # Initialize dict to store Backbone samples per year.
for (year, agg) in tsam_dict.items():
    sample_dict.setdefault(year, dict()) # Initialize empty dict for each year.
    # First timestep index of each sample, directly from cluster center indices.
    samples = {
        val * hoursPerPeriod + 1: ind
        for (val, ind) in zip_longest(agg.clusterCenterIndices, agg.clusterPeriodIdx)
    }
    # Calculate sample weights based on the cluster occurrence number.
    sample_weights = agg.clusterPeriodNoOccur
    normalized_sample_weights = {ind: val / sum(sample_weights.values()) for (ind, val) in sample_weights.items()}
    # Order samples by starting index and fetch the corresponding lengths !!!
    samples = [
        ("s" + f"{i}".zfill(3), val, sample_weights[ind], normalized_sample_weights[ind])
        for i, (val, ind) in enumerate(sorted(samples.items()))
    ]
    # Figure out the order the samples occur during the year
    # This is not used at the moment, see note below.
    #sample_order = agg.clusterOrder # Order of samples representing the raw data (year).
    
    # Record sample settings per year.
    sample_dict[year]["number_of_samples"] = noTypicalPeriods
    sample_dict[year]["representative_period_length_in_hours"] = hoursPerPeriod
    sample_dict[year]["sample_start_and_weight"] = samples

# How do we actually want to represent the year?
# If we use the periods where they actually occur (clusterCenterIndices),
# the weights won't match the outcome desired by TSAM when fed to Backbone.
# If we instead aim to match the weights, the periods cannot occur where Backbone needs them to.
# Then again, we need to place the samples where they are to get the correct data
# into Backbone (agg.clusterCenterIndices), so I guess we can't really consider the order?

In [ ]:
# Example output from sample_dict

sample_dict["1990"]

## 5. Output Backbone settings.

Finally, we need to output the representative periods for Backbone.
Ideally, this need to be done using the `weather_year` as the selector.
I suppose the easiest (if not the cleanest) way is to create a separate file per year,
similar to what is done with the timeseries.
Essentially, we'll want to autocreate a `samples_<year>.inc` or similar,
which can then be read in `investInit.gms` based on the given `climateYear`.

In [ ]:
## Write `samples_<year>.inc` files containing autogenerated sample settings.
# This is essentially autogenerated GAMS code...

for (year, sample_data) in sample_dict.items():
    with open(input_folder_path + f"samples_{year}.inc", 'w') as file:
        # Frontmatter
        file.write("* Automatically generated sample settings from `representative-period-processing.ipynb`\n")
        file.write(f"* {datetime.now().isoformat()}\n")
        # Number of samples
        file.write("\n* Number of samples used by the model \n")
        file.write(f"mSettings('invest', 'samples') = {sample_data['number_of_samples']};\n")
        # Clearing initial and central samples.
        file.write("\n* Clear initial and central samples \n")
        file.write("ms_initial('invest', s)$(ord(s) <= mSettings('invest', 'samples')) = yes;\n")
        file.write("ms_central('invest', s) = no;\n")
        # Declare equal probabilities for all samples.
        file.write("\n* Equal probability for all samples \n")
        file.write("p_msProbability(ms_initial) = 1;\n")
        # Define sample properties
        file.write("\n* Define sample properties \n")
        for (sample, start, weight, nweight) in sample_data['sample_start_and_weight']:
            file.write(f"* Sample {sample}\n")
            file.write(f"msStart('invest', '{sample}') = {start};\n")
            file.write(f"msEnd('invest', '{sample}') = {start + sample_data["representative_period_length_in_hours"]};\n")
            file.write(f"p_msWeight('invest', '{sample}') = {weight};\n")
            file.write(f"p_msAnnuityWeight('invest', '{sample}') = {nweight};\n")

### Using the autogenerated Backbone sample settings

Essentially, what the above code does is automatically output Backbone sample settings.
These are the settings typically given in e.g. `investInit.gms` or similar.
The automatically generated `samples_<year>.inc` files look something like this:

```gams
* Automatically generated sample settings from `representative-period-processing.ipynb`
* 2025-10-08T14:37:24.128610

* Number of samples used by the model 
mSettings('invest', 'samples') = 12;

* Clear initial and central samples 
ms_initial('invest', s)$(ord(s) <= mSettings('invest', 'samples')) = yes;
ms_central('invest', s) = no;

* Equal probability for all samples 
p_msProbability(ms_initial) = 1;

* Define sample properties 
* Sample s000
msStart('invest', 's000') = 169;
msEnd('invest', 's000') = 193;
p_msWeight('invest', 's000') = 16;
p_msAnnuityWeight('invest', 's000') = 0.043835616438356165;
* Sample s001
msStart('invest', 's001') = 913;
msEnd('invest', 's001') = 937;
p_msWeight('invest', 's001') = 43;
p_msAnnuityWeight('invest', 's001') = 0.1178082191780822;
* Sample s002
etc...
```

Currently, the autogeneration assumes all-initial no-central samples
_(not completely sure what they mean)_, as well as an equal probability across all samples.
You will need to tweak the above code or the output files manually
in case you want to change these settings.
The number of samples and their properties are based on the TSAM representatives.

>**NOTE!**
>The output samples are ordered based on their `msStart`!
>Essentially, samples `s000`, `s001`, `s002`, etc. proceed sequentially
>through the time steps, but will likely have "gaps" between each other.

**In order to plug these into your model, you'll need to load them in your `investInit.gms`
or similar.**
_(See e.g. `temp_investInit.gms` in the Backbone repository)_
Below an example for how to read the autogenerated sample settings into
your model based on the given `%input_dir%` and `%climateYear%` command line
arguments already used by the NE-model to select weather years from the data:

```gams
...
* =============================================================================
* --- Model Time Structure ----------------------------------------------------
* =============================================================================

* --- Define Samples ----------------------------------------------------------

    $$include '%input_dir%\samples_%climateYear%.inc' // THIS IS THE KEY ROW
    // Currently, the NE-model risks infeasibility if a sample starts at 1.
    // Temporary fix to delay by 1 hour.
    msStart(ms_initial)${msStart(ms_initial) = 1} = 2;

* --- Define Time Step Intervals ----------------------------------------------
...
```

At the time of writing, the NE-model has a peculiar issue where hydropower reservoir
initial states might cause an infeasibility IF they are released AND the first
sample starts at the first available time step.
The above code contains a manual bypass, where any sample starting on the first
time step is forcibly delayed by one step to avoid the possible infeasibility.
While it is quite rare for TSAM to begin a sample at the first available time step,
it can still occur _(and did in fact occur in my testing)_.

It should also be mentioned that depending on the type of modelling you're doing,
you might have to adapt other parts of your Backbone inputs to accommodate
the potentially varying properties of the samples.
I.e. make sure the `timeAndSamples.inc` has a sufficient number of base sample indices.
Additionally, defining `gnss_bound`s for how the samples bridge to themselves or each
other might require tweaking.
The `modelsInit.gms` in the `gg-smr-scenarios` branch of
[my forked NE-model](https://github.com/Tasqu/north_european_model/tree/gg-smr-scenarios)
contains potential examples for how these might be scripted.

# TODO?

Interesting idea by Niina: First reduce the number of years by selecting "representative years", then represent them using "representative periods".
This could significantly reduce the computational burden to represent multiple years.
However, we would likely want to include different weather years for the investments, so I'm not sure if TSAM can handle that.